In [1]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Cargar los datos previamente procesados
df_departamentos = pd.read_csv('departamentos_peru.csv')
df_temperaturas = pd.read_csv('temperatura_promedio_2020_2024.csv')

# Convertir temperaturas a formato largo
df_long = df_temperaturas.melt(id_vars='Departamento', var_name='Fecha', value_name='Temperatura')
df_long[['Mes', 'Año']] = df_long['Fecha'].str.extract(r'([A-Za-z]+)-(\d{4})')
df_long['Mes'] = pd.Categorical(df_long['Mes'], categories=[
    'Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'], ordered=True)
df_long['Año'] = df_long['Año'].astype(int)

# Promedio por departamento y mes
df_promedio_mes = df_long.groupby(['Departamento', 'Mes'])['Temperatura'].mean().reset_index()

# Clasificación por clima
def clasificar_temperatura(temp):
    if temp < 10:
        return 'Muy frío'
    elif temp < 15:
        return 'Frío'
    elif temp < 20:
        return 'Templado'
    elif temp < 25:
        return 'Caluroso'
    else:
        return 'Muy caluroso'

df_promedio_mes['Clima'] = df_promedio_mes['Temperatura'].apply(clasificar_temperatura)

# Unir con descripciones
df_resultado = pd.merge(df_promedio_mes, df_departamentos, left_on='Departamento', right_on='departamento', how='left')
df_resultado = df_resultado.drop(columns='departamento')
df_resultado_filtrado = df_resultado.dropna(subset=['descripcion'])

# Widgets
mes_selector = widgets.Dropdown(
    options=df_resultado_filtrado['Mes'].unique().tolist(),
    description='Mes:',
    style={'description_width': 'initial'}
)

temperatura_input = widgets.FloatSlider(
    value=20.0,
    min=0,
    max=35,
    step=0.5,
    description='Temp. deseada (°C):',
    style={'description_width': 'initial'}
)

boton_buscar = widgets.Button(description="Buscar recomendaciones", button_style='success')
salida = widgets.Output()

# Función de búsqueda
def buscar_recomendaciones(b):
    with salida:
        clear_output()
        mes = mes_selector.value
        temp_deseada = temperatura_input.value
        rango_inferior = temp_deseada - 5
        rango_superior = temp_deseada + 5

        resultados = df_resultado_filtrado[
            (df_resultado_filtrado['Mes'] == mes) &
            (df_resultado_filtrado['Temperatura'].between(rango_inferior, rango_superior))
        ][['Departamento', 'Temperatura', 'Clima', 'descripcion']].sort_values(by='Temperatura')

        if resultados.empty:
            print(f"No se encontraron departamentos con temperatura cercana a {temp_deseada} °C en {mes}.")
        else:
            print(f"Departamentos recomendados para {mes} con temperatura entre {rango_inferior} °C y {rango_superior} °C:")
            display(resultados)

boton_buscar.on_click(buscar_recomendaciones)

# Mostrar la interfaz
display(widgets.VBox([mes_selector, temperatura_input, boton_buscar, salida]))


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_7484\1975027286.py:17: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_promedio_mes = df_long.groupby(['Departamento', 'Mes'])['Temperatura'].mean().reset_index()


In [2]:
print(df_resultado_filtrado)

    Departamento  Mes  Temperatura         Clima  \
0       Amazonas  Ene       15.198      Templado   
1       Amazonas  Feb       15.414      Templado   
2       Amazonas  Mar       15.250      Templado   
3       Amazonas  Abr       14.990          Frío   
4       Amazonas  May       15.106      Templado   
..           ...  ...          ...           ...   
271      Ucayali  Ago       26.552  Muy caluroso   
272      Ucayali  Sep       27.220  Muy caluroso   
273      Ucayali  Oct       27.326  Muy caluroso   
274      Ucayali  Nov       26.388  Muy caluroso   
275      Ucayali  Dic       26.148  Muy caluroso   

                                           descripcion  
0    El departamento de Amazonas es reconocido por ...  
1    El departamento de Amazonas es reconocido por ...  
2    El departamento de Amazonas es reconocido por ...  
3    El departamento de Amazonas es reconocido por ...  
4    El departamento de Amazonas es reconocido por ...  
..                               

In [6]:
df_resultado_filtrado.to_csv("datos_recomendacion_turismo.csv", index=False)